## What was discovered?
* is_tourney (remark when game was in regular or tourney phase) variable doesnt matter according to feature importance and also we will predict only tourney phase games.
* to addres the problem of data leakage I removed scores from games (before game we dont know which teams shoot how many points) and also for every team i retrieve seed values. But if it is regular season i only have information about previous ones. So for regular season I retrieve seed from previous season and for tourney phase i retrieve information from current season.
* team coaches can have also some important informations (some coaches are better than other). But during the season team coache can be changed. So apart classic merge I have to check also if in particular row daynum is smaller than last day and bigger than first day of coach term.
* DayNum feature can carry some information. But in this contest we do not have information about matches (we have to predict every possible matchup). Feature importance indicate it is important feature.

## Which features was added?
* seed value for every team
* difference between seed of one team and another
* team coach information
* duration of teams presence in Division I


## Load libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import brier_score_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from custom_utils import transform_data, enrich_data, feature_importance
from IPython.display import display
from sklearn.decomposition import PCA
from collections import defaultdict

## Load and prepare data

In [4]:
wregularseason = pd.read_csv("data/WRegularSeasonCompactResults.csv")
wtourneyseason = pd.read_csv("data/WNCAATourneyCompactResults.csv")

mregularseason = pd.read_csv("data/MRegularSeasonCompactResults.csv")
mtourneyseason = pd.read_csv("data/MNCAATourneyCompactResults.csv")

mseason = pd.read_csv("data/MSeasons.csv")
wseason = pd.read_csv("data/WSeasons.csv")

mrankings = pd.read_csv("data/MMasseyOrdinals.csv")

wprep = pd.concat([transform_data(wregularseason), transform_data(wtourneyseason)], axis=0).reset_index(drop=True)
mprep = pd.concat([transform_data(mregularseason), transform_data(mtourneyseason)], axis=0).reset_index(drop=True)

In [5]:
mprep_enh = enrich_data(mprep, "M")
wprep_enh = enrich_data(wprep, "W")
wprep_enh = wprep_enh.loc[wprep_enh["Season"] >= 2003]
mprep_enh = mprep_enh.loc[mprep_enh["Season"] >= 2003]
display(wprep_enh.head())
mprep_enh.head()

c:\Users\urbii\Desktop\Projekty\Forecasting-NCAA-BB\venv\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\urbii\Desktop\Projekty\Forecasting-NCAA-BB\venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,Season,DayNum,WTeamID,LTeamID,Result,is_tournament,SeedW,SeedL,SeedDiff
21389,2003,6,3256,3403,1,0,5,4,1
21390,2003,6,3397,3328,1,0,2,1,1
21391,2003,11,3161,3362,1,0,7,16,-9
21392,2003,11,3210,3150,1,0,16,16,0
21393,2003,11,3243,3123,1,0,3,16,-13


,Season,DayNum,WTeamID,LTeamID,Result,is_tournament,SeedW,SeedL,WCoachName,LCoachName,Duration,SeedDiff
74048,2003,10,1104,1328,1,0,2,2,1000,872,19.0,0
74049,2003,10,1272,1393,1,0,16,16,799,712,19.0,0
74050,2003,11,1266,1437,1,0,5,16,1523,655,19.0,-11
74051,2003,11,1296,1457,1,0,16,16,1301,577,19.0,0
74052,2003,11,1400,1208,1,0,6,3,1275,734,19.0,3


In [ ]:
dict_of_rank = defaultdict(list)
for index, row in mprep_enh.iterrows():
    try:
        daynums = mrankings[(mrankings["Season"] == row["Season"]) & (mrankings["TeamID"] == row["WTeamID"])].RankingDayNum.unique()
        closestdaynum = daynums[daynums <= row["DayNum"]][-1]
        dict_of_rank["RankingW"].append(np.median(mrankings[(mrankings["Season"] == row["Season"]) & (mrankings["TeamID"] == row["WTeamID"]) & (mrankings["RankingDayNum"] == closestdaynum)].OrdinalRank).item())
    except:
        dict_of_rank["RankingW"].append(0)

    try:
        daynums = mrankings[(mrankings["Season"] == row["Season"]) & (mrankings["TeamID"] == row["LTeamID"])].RankingDayNum.unique()
        closestdaynum = daynums[daynums <= row["DayNum"]][-1]
        dict_of_rank["RankingL"].append(np.median(mrankings[(mrankings["Season"] == row["Season"]) & (mrankings["TeamID"] == row["LTeamID"]) & (mrankings["RankingDayNum"] == closestdaynum)].OrdinalRank).item())
    except:
        dict_of_rank["RankingL"].append(0)

## Test models on different features

### Divide dataset on test and train sets

In [4]:
N_COMPONENTS = 3
pca = PCA(n_components=N_COMPONENTS)

feature_labels = [f"feature_{i}" for i in range(1, N_COMPONENTS + 1)]

mprep_enh = mprep_enh.loc[mprep_enh["Season"] > 1985]
wprep_enh = wprep_enh.loc[wprep_enh["Season"] > 1998]
X_featm = mprep_enh.drop(["Result", "DayNum"], axis=1)
ym = mprep_enh.Result

X_featw = wprep_enh.drop(["Result", "DayNum"], axis=1)
yw = wprep_enh.Result

X_trainm, X_testm, y_trainm, y_testm = train_test_split(X_featm, ym, test_size=0.1)
X_trainw, X_testw, y_trainw, y_testw = train_test_split(X_featw, yw, test_size=0.1)

X_trainm_pca = pd.DataFrame(pca.fit_transform(X_trainm), columns=feature_labels)
X_testm_pca = pd.DataFrame(pca.transform(X_testm), columns=feature_labels)

X_trainw_pca = pd.DataFrame(pca.fit_transform(X_trainw), columns=feature_labels)
X_testw_pca = pd.DataFrame(pca.transform(X_testw), columns=feature_labels)

### Without PCA

In [5]:
rf_clfm = RandomForestClassifier()
rf_clfm.fit(X_trainm, y_trainm)

rf_clfw = RandomForestClassifier()
_ = rf_clfw.fit(X_trainw, y_trainw)

In [6]:
pred_probsm = rf_clfm.predict_proba(X_testm)[:, 1]
pred_probsw = rf_clfw.predict_proba(X_testw)[:, 1]

print(f"Brier score for mens data: {np.round(brier_score_loss(y_testm, pred_probsm), 3).item()}")
print(f"Brier score for womens data: {np.round(brier_score_loss(y_testw, pred_probsw), 3).item()}")

Brier score for mens data: 0.25
Brier score for womens data: 0.233


In [12]:
print("For men's games")
display(feature_importance(rf_clfm, mprep_enh.drop("DayNum", axis=1)))

print("For women's games")
display(feature_importance(rf_clfw, wprep_enh.drop("DayNum", axis=1)))

For men's games


,Feature,Importance_clf,Importance_mi
0,Season,0.117652,0.001438
1,WTeamID,0.156088,0.022414
2,LTeamID,0.172326,0.021905
3,is_tournament,0.001348,0.001644
4,SeedW,0.025466,0.053569
5,SeedL,0.024011,0.054426
6,WCoachName,0.159619,0.032300
7,LCoachName,0.175787,0.033418
8,Duration,0.119032,0.000467
9,SeedDiff,0.048671,0.033102


For women's games


,Feature,Importance_clf,Importance_mi
0,Season,0.117984,0.000607
1,WTeamID,0.387779,0.028062
2,LTeamID,0.372157,0.029269
3,is_tournament,0.001315,0.000509
4,SeedW,0.029644,0.047571
5,SeedL,0.030592,0.045157
6,SeedDiff,0.060530,0.051971


### With PCA

In [15]:
rf_clfm = RandomForestClassifier()
rf_clfm.fit(X_trainm_pca, y_trainm)

rf_clfw = RandomForestClassifier()
_ = rf_clfw.fit(X_trainw_pca, y_trainw)


pred_probsm = rf_clfm.predict_proba(X_testm_pca)[:, 1]
pred_probsw = rf_clfw.predict_proba(X_testw_pca)[:, 1]

print(f"Brier score for mens data: {np.round(brier_score_loss(y_testm, pred_probsm), 3).item()}")
print(f"Brier score for womens data: {np.round(brier_score_loss(y_testw, pred_probsw), 3).item()}")

Brier score for mens data: 0.278
Brier score for womens data: 0.244


In [16]:
wdatatrain = pd.concat([X_trainw_pca, y_trainw.reset_index(drop=True)], axis=1)
mdatatrain = pd.concat([X_trainm_pca, y_trainm.reset_index(drop=True)], axis=1)

wdatatest = pd.concat([X_testw_pca, y_testw.reset_index(drop=True)], axis=1)
mdatatest = pd.concat([X_testm_pca, y_testm.reset_index(drop=True)], axis=1)

wdata = pd.concat([wdatatrain, wdatatest], axis=0).reset_index(drop=True)
mdata = pd.concat([mdatatrain, mdatatest], axis=0).reset_index(drop=True)

print("For men's games")
display(feature_importance(rf_clfm, mdata))

print("For women's games")
display(feature_importance(rf_clfw, wdata))

For men's games


,Feature,Importance_clf,Importance_mi
0,feature_1,0.332923,0.069217
1,feature_2,0.333765,0.068002
2,feature_3,0.333312,0.038686


For women's games


,Feature,Importance_clf,Importance_mi
0,feature_1,0.339288,0.086421
1,feature_2,0.344058,0.117547
2,feature_3,0.316654,0.031244
